# HOTA evaluation


## Prepare datasets 

In this script, we are mainly moving data into the right directory and format, for HOTA evaluation. Unfortunately this is currently only possible within the [TrackEval](https://github.com/JonathonLuiten/TrackEval/) package.

To run the script without modifications the data needs to be in the following structure - otherwise it was to be adapted.

gt_folder \
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|-------Vid1 \
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|-----labels_with_ids \
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|------frame_000000.txt\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|------frame_000001.txt\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|------Vid2...\
\
\
pred_folder\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|------Vid1\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|------model1\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|------results.txt\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|------model2\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|------results.txt\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|------Vid2...
            


In [1]:
import os
import re
import sys
import numpy as np
import shutil
import csv

In [2]:
#where are the both datasets/ where is TrackEval located
animal = "lemur" # or "lemur"

base_path_gt = "/path/to/gt/"+animal+"_videos_eval/"
base_path_predictions = "/path/to/predictions/"
base_path_TrackEval = "/path/to/folder/TrackEval"
experiment_name = animal+"_tracking_3seeds" 
name_in_videos =  "Eval\d+" if animal=="lemur" else "vid_" # some string that all videos have in common


### Ground truth

In [8]:
def gt_files_to_dict(folder_path, scale_size = (1920, 1080)):
    '''
    input: path to a video, to the folder which contains the label files (in the format that there is 
    one label file per frame)
    output: dictionary containing all the frames and bboxes
    '''
    width, height = scale_size
    data_names = os.listdir(folder_path)
    data_names = [name for name in data_names if re.search(".txt",name)]
    frame_nr = []
    class_name = []
    id_nr = []
    bbox_xywh = []
    for data_name in data_names:
        maca_data = open(os.path.join(folder_path,data_name)).read().strip().split("\n")
        if maca_data == [""]:
            continue
        maca_data = [[float(nr) for nr in row.strip().split(" ")] for row in maca_data]
        frame_name = data_name.split(".")[0]
        #-1 that mot also starts at 0
        frame_nr_to_append = int(re.findall("[0-9]+",frame_name)[0])
        for row in maca_data:
            frame_nr.append(frame_nr_to_append)
            class_name.append(row[0])
            id_nr.append(row[1])
            #to upper left corner from middle 
            row[2] = (float(row[2]) - float(row[4])/2) * width
            row[3] = (float(row[3]) - float(row[5])/2) * height
            row[4] = float(row[4]) * width
            row[5] = float(row[5]) * height
            row[2:6] = [round(x,2) for x in row[2:6]]
            bbox_xywh.append(row[2:6])
    #now bring them in the right order of the frames for sure:
    order_to_follow = np.argsort(frame_nr)
    frame_nr = [frame_nr[i] for i in order_to_follow]
    id_nr = [id_nr[i] for i in order_to_follow]
    bbox_xywh = [bbox_xywh[i] for i in order_to_follow]
    class_name = [class_name[i] for i in order_to_follow]
    return {"frame_name": frame_nr, "id_nr": id_nr, 
            "bbox_xywh": bbox_xywh, "class_name": class_name}

def dict_to_file(path, dict_to_write):
    """
    Write a txt file with all the bboxes for the given dictionary (in the MOTChallenge format)

    path: path to write the file
    mot_dict: dictionary with all the information (coming from the function above)
    """
    frame_nr = dict_to_write["frame_name"]
    id_nr = dict_to_write["id_nr"]
    bbox_xywh = dict_to_write["bbox_xywh"]
    confidence_score = dict_to_write["confidence_score"]
    with open(path, "w") as f:
        for i in range(len(frame_nr)):
            row = [str(frame_nr[i]+1), str(int(id_nr[i])), *[str(round(x, 2)) for x in bbox_xywh[i]], str(round(confidence_score[i], 2)),str(1),str(1)]
            f.write(",".join(row) + "\n")



In [9]:

videos = os.listdir(base_path_gt)
videos = [vid for vid in videos if re.search(name_in_videos, vid)]



In [10]:

for video in videos:
    infos = gt_files_to_dict(os.path.join(base_path_gt, video, "labels_with_ids"), scale_size = (1920, 1080))
    print(video)
    infos["confidence_score"] = [1 for _ in infos["id_nr"]]
    if not os.path.exists(os.path.join(base_path_gt, video, "gt")):
        os.makedirs(os.path.join(base_path_gt, video, "gt"))
    dict_to_file(os.path.join(base_path_gt, video, "gt", "gt.txt"), infos)



Eval19
Eval9
Eval11
Eval15
Eval5
Eval23
Eval2
Eval12
Eval16
Eval6
Eval20
Eval7
Eval21
Eval17
Eval13
Eval3
Eval8
Eval18
Eval4
Eval22
Eval14
Eval10


In [11]:

for video in videos:
    #seqLength = len(os.listdir(os.path.join(base_path, video, "labels_with_ids")))
    seqLength = float('-inf')
    with open(os.path.join(base_path_gt, video, "gt", "gt.txt"), 'r') as file:
        for line in file:
            first_column = int(line.split(',')[0])
            seqLength = max(seqLength, first_column)

    with open(os.path.join(base_path_gt, video, "seqinfo.ini"), "w") as to_write:
        to_write.write("[Sequence]\n")
        to_write.write("; name=MOT16-01\n")
        to_write.write(f"; imDir={video}/images\n")
        to_write.write("frameRate=30\n")
        to_write.write(f"seqLength={seqLength}\n")
        to_write.write("imWidth=1920\n")
        to_write.write("imHeight=1080\n")
        to_write.write("imExt=.jpg\n")


In [12]:
base_path_TrackEval_gt = os.path.join(base_path_TrackEval, "data", "gt")
base_path_gt_dataset = os.path.join(base_path_TrackEval_gt, experiment_name)
    
#create the folder structure and assume that mot annotation for macaque video is already created with the above cells
if not os.path.exists(base_path_gt_dataset):
    os.makedirs(base_path_gt_dataset)


#copy groundtruth
seq_names = [seq_name for seq_name in os.listdir(base_path_gt) if re.search(name_in_videos,seq_name )]

# Copy the ground truth files for each sequence to the base_path_gt directory
for seq_name in seq_names:
    # Get the full path to the ground truth file
    gt_file = os.path.join(base_path_gt, seq_name, "gt", "gt.txt")
    info_file = os.path.join(base_path_gt, seq_name, "seqinfo.ini")
    
    # Create the directory for the sequence in base_path_gt if it doesn't exist
    seq_dir = os.path.join(base_path_gt_dataset, seq_name,"gt")
    if not os.path.exists(seq_dir):
        os.makedirs(seq_dir)
    
    # Copy the ground truth file to the sequence directory in base_path_gt
    shutil.copy(gt_file, seq_dir)
    shutil.copy(info_file, os.path.dirname(seq_dir))

# create seqnames
seqnames_path = os.path.join(os.path.dirname(base_path_gt_dataset),"seqmaps")
if not os.path.exists(seqnames_path):
    os.makedirs(seqnames_path)

# for some reason it often doesn't work on the first try, run this cell twice or several times
    #'SPLIT_TO_EVAL': 'all'

with open(os.path.join(seqnames_path, experiment_name+".txt"), "w") as seqnames:
    seqnames.write("name\n")
    for seq_name in seq_names:
        seqnames.write(seq_name+"\n")


### Move predictions

In [3]:
#tracker_name = "macaquepose_1_150"

models = ['model_50', 'model_100', 'model_150', 'model_200']
confs = ['0.01', '0.02', '0.04', '0.1', '0.2', '0.4']
dets = ['0.4', '0.5', '0.6']
assocs = ['0.7', '0.8', '0.9']
news = ['0.5', '0.6', '0.7', '0.8']
propious = ['0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1']
datasets = ['imagenet', 'macaquecp', 'nopretrain', 'macaquecpw']
lr = ['_5e-5']
seeds = ['1','2','3']
epochs = list(range(10, 201, 10))
methods = ['singlekalman', 'doublekalman']

#tracker_names = ["macaques_" + model + "_" + str(epoch) for model in models for epoch in epochs]
#tracker_names = [animal+"s_"  + dataset + "_" + str(i) + l for i in range(10, 201,10) for dataset in datasets for l in lr] #lemur_tracking
tracker_names = [animal+"s_"  + dataset + "_" + str(i) for i in range(1,4) for dataset in datasets] #lemur_tracking_3seeds
#tracker_names = [animal+"s_"+str(iou) for iou in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]]
#tracker_names = [animal+"s_" + conf + "_" + det + "_" + assoc + "_" + propiou + "/" + method for conf in confs for det in dets for assoc in assocs for propiou in propious for method in methods]

#tracker_names = [animal+"s_" + conf + "_" + det + "_" + new +"/singlekalman" for conf in confs for det in dets for new in news]

for tracker_name in tracker_names:
    
    base_path_trackers = os.path.join(base_path_TrackEval, "data", "trackers")
    base_path_tracker = os.path.join(base_path_trackers, experiment_name, tracker_name.replace("/", "_"), "data")
    

    if not os.path.exists(os.path.join(base_path_trackers, experiment_name, tracker_name.replace("/", "_"), "data")):
        os.makedirs(base_path_tracker)


    #copy tracker results
    #If this cannot be searched by name, you can also manually put it
    seq_names = [seq_name for seq_name in os.listdir(os.path.join(base_path_predictions, tracker_name)) if re.search(name_in_videos, seq_name )]
    seq_dir = os.path.join(base_path_tracker)

    for seq_name in seq_names:
        # Get the full path to the ground truth file
        #print(os.path.join(base_path_macaque_results, seq_name, tracker_name, "results.txt"))
        #det_file = os.path.join(base_path_predictions, tracker_name, seq_name, seq_name + ".txt")
        det_file = os.path.join(base_path_predictions, tracker_name, seq_name)

        # in case that there are several classes, we multiply each object by 100 * class_number to prevent double IDs
        with open(det_file, 'r') as file:
            lines = file.readlines()

        # Process each line
        for i, line in enumerate(lines):
            # Split the line into elements
            elements = line.strip().split(", ")

            # Perform the desired calculations
            second_element = int(elements[1])
            last_element = int(elements[-1])
            updated_value = second_element + last_element * 100

            # Update the line with the new value
            elements[1] = str(updated_value)
            updated_line = ','.join(elements)+"\n"

            # Update the list of lines
            lines[i] = updated_line

        # Write the updated lines back to the file
        with open(os.path.join(seq_dir, seq_name), 'w') as file:
            file.writelines(lines)
        


## Consolidate results

This happens after running python TrackEval/run_HOTA_evaluation.py --BENCHMARK [insert name of experiment]

In [9]:
experiment_name = animal+"_tracking_3seeds"

folder_path = "../TrackEval/data/trackers/"+ experiment_name
output_file = os.path.join(folder_path,"..","..", "summary_" + experiment_name + ".txt")

In [10]:
with open(output_file, 'w', newline='') as outfile:
    writer = csv.writer(outfile)

    # Initialize a flag to track if the header has been written
    header_written = False

    # Iterate over the subfolders in the specified directory
    for root, dirs, files in os.walk(folder_path):
        dirs.sort()
        # Look for the box_detailed.csv file in each subfolder
        if 'macaque_summary.txt' in files:
            file_path = os.path.join(root, 'macaque_summary.txt')

            # Read the contents of the file and write them to the output CSV
            with open(file_path, 'r') as infile:
                reader = csv.reader(infile, delimiter=" ")
                header = next(reader)

                # Check if the header needs to be written
                if not header_written:
                    header.insert(0, 'metric')
                    writer.writerow(header)
                    header_written = True

                # Write the remaining rows
                for row in reader:
                    row.insert(0, str(root.split("/")[-1]))
                    writer.writerow(row)